In [1]:
]activate ..; instantiate; st

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


┌ Warning: Some registries failed to update:
│     — /home/gridsan/dyuret/.julia/registries/General — failed to fetch from repo
└ @ Pkg.API /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.0/Pkg/src/API.jl:157


    Status `~/Klutz.jl/Project.toml`
  [3a865a2d] CuArrays v0.8.1
  [587475ba] Flux v0.6.8
  [1902f260] Knet v1.1.1


In [2]:
# Uncomment this to get Knet profiling info at the end:
# ENV["KNET_TIMER"] = ENV["AUTOGRAD_TIMER"] = "true"
# using Pkg; Pkg.build("AutoGrad"); Pkg.build("Knet")

  Building SpecialFunctions → `~/.julia/packages/SpecialFunctions/fvheQ/deps/build.log`
  Building SpecialFunctions → `~/.julia/packages/SpecialFunctions/fvheQ/deps/build.log`
  Building CodecZlib ───────→ `~/.julia/packages/CodecZlib/DAjXH/deps/build.log`
  Building Knet ────────────→ `~/.julia/packages/Knet/3lzCR/deps/build.log`


In [3]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated, partition
using CuArrays
using Knet: Knet, AutoGrad, conv4, pool, KnetArray
Knet.gpu()

┌ Info: Recompiling stale cache file /home/gridsan/dyuret/.julia/compiled/v1.0/Flux/QdkVy.ji for Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1187
┌ Info: Recompiling stale cache file /home/gridsan/dyuret/.julia/compiled/v1.0/CuArrays/7YFE0.ji for CuArrays [3a865a2d-5b23-5a0f-bc46-62713ec82fae]
└ @ Base loading.jl:1187
┌ Info: Recompiling stale cache file /home/gridsan/dyuret/.julia/compiled/v1.0/Knet/f4vSz.ji for Knet [1902f260-5fb4-5aff-8c31-6271790ab950]
└ @ Base loading.jl:1187


0

In [4]:
# Implement Chain, Conv and Dense in Knet
struct kChain; layers; kChain(ls::Tuple)=new(ls); end
kChain(ls...)=kChain(ls)
(c::kChain)(x) = (for l in c.layers; x = l(x); end; x)
struct kDense; w; b; f; end
kDense(nx::Int,ny::Int,fn=identity)=kDense(param(ny,nx),param0(ny),fn)
(d::kDense)(x) = d.f.(d.w * Knet.mat(x) .+ d.b)
struct kConv; w; b; f; end
kConv(w1,w2,cx,cy,fn=identity)=kConv(param(w1,w2,cx,cy), param0(1,1,cy,1), fn)
(f::kConv)(x) = pool(f.f.(conv4(f.w,x) .+ f.b))

In [5]:
# Load data
imgs = MNIST.images()
labels = onehotbatch(MNIST.labels(), 0:9)
train = [(cat(float.(imgs[i])..., dims = 4), labels[:,i])
         for i in partition(1:60_000, 1000)]
train = gpu.(train)
klabels = MNIST.labels() .+ 1
ktrain = [(KnetArray{Float32}(cat(float.(imgs[i])..., dims = 4)), klabels[i])
          for i in partition(1:60_000, 1000)]
summary.((train[1]..., ktrain[1]...))

("28×28×1×1000 CuArray{Float32,4}", "10×1000 Flux.OneHotMatrix{CuArray{Flux.OneHotVector,1}}", "28×28×1×1000 KnetArray{Float32,4}", "1000-element Array{Int64,1}")

In [11]:
# Run this several times to get timing for Flux:
m = Chain(
  Conv((2,2), 1=>16, relu),
  x -> maxpool(x, (2,2)),
  Conv((2,2), 16=>8, relu),
  x -> maxpool(x, (2,2)),
  x -> reshape(x, :, size(x, 4)),
  Dense(288, 10), softmax) |> gpu

loss(x, y) = crossentropy(m(x), y)
accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))
opt = ADAM(params(m))

X,Y = train[1]
@show loss(X, Y), accuracy(X, Y)
@time Flux.train!(loss, train, opt)
@show loss(X, Y), accuracy(X, Y);

(loss(X, Y), accuracy(X, Y)) = (2.3025045f0 (tracked), 0.117)
  0.861841 seconds (236.63 k allocations: 14.045 MiB, 18.61% gc time)
(loss(X, Y), accuracy(X, Y)) = (1.4854774f0 (tracked), 0.673)
